In [1]:
import numpy as np
import torch
import torch.nn as nn

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, input_channel, output_channel, stride=1, downsample=None):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channel, output_channel, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(output_channel),
            nn.ReLU()   
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(output_channel, output_channel, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(output_channel)
        )

        self.relu = nn.ReLU()
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.conv2(out)

        if self.downsample:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


In [55]:
class ResNet(nn.Module):
    def __init__(self, input_channel, block, layers, num_classes=47):
        super().__init__()
        
        self.inplanes = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channel, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer0 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool1 = nn.AvgPool2d(kernel_size=7, stride=1)
        self.avgpool2 = nn.AvgPool2d(kernel_size=3, stride=1)
        
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, n_blocks, stride):
        
        dowsample = None
        if stride != 1 or self.inplanes != planes:
            dowsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes)
            )

        layer = []
        layer.append(block(self.inplanes, planes, stride, dowsample))
        self.inplanes = planes

        for _ in range(1, n_blocks):
            layer.append(block(self.inplanes, planes))
        
        return nn.Sequential(*layer)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.maxpool(out)
        out = self.layer0(out)
        out = self.layer1(out)
        out = self.layer2(out)
        # out = self.layer3(out)

        out = self.avgpool2(out)
        # out = out.reshape(out.size(0), -1)
        # out = self.fc(out)

        return out



In [56]:
model = ResNet(block=ResidualBlock, input_channel=1, layers=[2, 2, 2, 2])

In [57]:
model.eval()
a = torch.randn(1, 1, 28, 28)
out = model(a)

RuntimeError: Given input size: (256x2x2). Calculated output size: (256x0x0). Output size is too small

In [54]:
out.size()

torch.Size([1, 256, 2, 2])